In [ ]:
## Initialize Notebook Loader
from src import interface
import requests

In [ ]:
single_match = interface.request_match_data("id", id=6002791667)
print(single_match)

In [ ]:
parsed_matches = interface.request_match_data("parsed")
print(parsed_matches)

In [ ]:
import json
import pandas as pd
import numpy as np

matches_json = json.loads(parsed_matches)


In [ ]:
key = 'pings'
list_player_list_1 = []

for j in range(len(matches_json)):
    just_players = []
    plyr = matches_json[j]
    for i in range(len(plyr['players'])):
        just_players.append({key:value for key, value in plyr['players'][i].items()})
    player_list = []
    player_list.append(matches_json[j]['match_id'])
    for i in range(len(plyr['players'])):
        player_list.append(just_players[i].get(key, 0)) # adds players to list 
    list_player_list_1.append(player_list)
    
df = pd.DataFrame.from_records(list_player_list_1)
df.columns = ['MatchId', 'Winner1','Winner2','Winner3','Winner4','Winner5','Loser1', 'Loser2', 'Loser3', 'Loser4', 'Loser5']

In [ ]:
print(df)


In [ ]:
df['LossPings']= df.iloc[:, -5:,].sum(axis=1)
df['WinPings']= df.iloc[:, 1:6].sum(axis=1)
df['CMP'] = np.where(df['WinPings'] >= df['LossPings'] ,1 , 0)
print(df)

In [ ]:
ttl_pings = df["WinPings"].append(df["LossPings"], ignore_index=True)
win_loss_ttl = pd.Series(np.repeat([1, 0],[100, 100]))
dft = pd.concat([ttl_pings, win_loss_ttl], axis = 1)
dft.columns = ["Pings", "Win"]


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.catplot(x = "CMP", y = "WinPings", data = df[["CMP", "WinPings"]])
ax = ax.axes.flatten()
ax[0].set_title("Team mit mehr Pings hat gewonnen")
ax[0].set_xlabel("Anzal der Pings im Gewinnerteam")
ax[0].set_ylabel("Anzahl der Pings")
ax2 = sns.catplot(x = "CMP", y = "LossPings", data = df[["CMP", "LossPings"]])
ax2 = ax2.axes.flatten()
ax2[0].set_title("Team mit mehr Pings hat verloren")
ax2[0].set_xlabel("Anzal der Pings im Verliererteam")
ax2[0].set_ylabel("Anzahl der Pings")

ax3 = sns.catplot(y = "Pings", x = "Win", data = dft)
ax3 = ax3.axes.flatten()
ax3[0].set_title("Pings vs Sieg/Niederlage")
ax3[0].set_xlabel("Sieg/Niederlage")
ax3[0].set_ylabel("Anzahl der Pings")
plt.show()

In [ ]:
import time

m = requests.get("https://api.opendota.com/api/players/118247684/matches")
mjson = m.json()

filter_keys_1 = ['match_id']
key = 'pings'

just_match_ids = []

for i in range(125):
    just_match_ids.append(mjson[i]['match_id'])

matches = []

for i in range(125):
    time.sleep(1)
    b = requests.get("https://api.opendota.com/api/matches/"+str(just_match_ids[i]))
    matches.append(b.json())

filter_keys = ['pings', 'win']
just_players = []

list_player_list_1 = []

for j in range(len(matches)):
    just_players = []
    plyr = matches[j]
    for i in range(len(plyr['players'])):
        just_players.append({key:value for key, value in plyr['players'][i].items() if key in filter_keys})
    player_list = []
    player_list.append(mjson[j]['match_id'])
    for i in range(len(plyr['players'])):
        p = just_players[i]
        player_list.append(p.get(key, 0))
    list_player_list_1.append(player_list)    

df = pd.DataFrame.from_records(list_player_list_1)
df.columns = ['MatchId', 'Winner1','Winner2','Winner3','Winner4','Winner5','Loser1', 'Loser2', 'Loser3', 'Loser4', 'Loser5']
df.head()



In [ ]:
filter_keys = ['assists', 'deaths', 'kills', 'pings', 'win']
player_kda = []
account_id = 118247684 

playerid_matches = matches[0]
for j in range(len(matches)):
    playerid_matches = matches[j]
    for i in range(len(playerid_matches['players'])):
        if playerid_matches['players'][i]['account_id'] == account_id:
            player_kda.append({key:value for key, value in playerid_matches['players'][i].items() if key in filter_keys})

df_kda = pd.DataFrame(player_kda)

In [ ]:
df_kda = df_kda.dropna()

In [ ]:
sns.lmplot(x = 'pings', y = 'kills', data = df_kda, hue = 'win', fit_reg = False)
plt.title("kills vs Pings")
plt.show()

In [ ]:
sns.lmplot(x = 'pings', y = 'deaths', data = df_kda, hue = 'win', fit_reg = False)
plt.title("deaths vs Pings")
plt.show()

In [ ]:
sns.lmplot(x = 'pings', y = 'assists', data = df_kda, hue = 'win', fit_reg = False)
plt.title("assists vs Pings")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

y = df_kda["win"]
X = df_kda.drop(["win"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)




In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)

clf = clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print("KNN", accuracy)